In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score

C:\Users\mique\AppData\Local\Temp\ipykernel_9092\872577557.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
data = pd.read_csv('../data/clean_data.csv')

In [3]:
X = data.drop('Severity', axis=1)
y = data['Severity']


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=1234)

dtc = RandomForestClassifier(random_state=1234)

dtc.fit(X_train, y_train)
y_pred = dtc.predict(X_test)

print(f'Accuracy: {accuracy_score(y_test, y_pred)}')
print(f'F1 Score: {f1_score(y_test, y_pred, average="weighted")}')

Accuracy: 0.8685634367055226
F1 Score: 0.8463690162936693
